In [1]:
import cv2
import numpy as np
import tensorflow
from tensorflow.keras.models import model_from_json
import operator
import time
import sys
import os
import matplotlib.pyplot as plt
from string import ascii_uppercase
from PIL import Image, ImageTk

mod=tensorflow.keras.models.load_model('model.h5')    



cap=cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 900)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 900)


def predict(test_image):
    test_image = cv2.resize(test_image, (128,128))
    result = mod.predict(test_image.reshape(1, 128, 128, 1))
   # result_dru = loaded_model_dru.predict(test_image.reshape(1 , 128 , 128 , 1))
    #result_tkdi = loaded_model_tkdi.predict(test_image.reshape(1 , 128 , 128 , 1))
    #result_smn = loaded_model_smn.predict(test_image.reshape(1 , 128 , 128 , 1))
    prediction={}
    prediction['blank'] = result[0][0]
    inde = 1
    for i in ascii_uppercase:
        prediction[i] = result[0][inde]
        inde += 1
    # LAYER 1
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
    current_symbol = prediction[0][0]
    return current_symbol
    

while(True):
    ok, frame = cap.read()
    if ok:
        cv2image = cv2.flip(frame, 1)
        x1 = int(0.5*frame.shape[1])
        y1 = 10
        x2 = frame.shape[1]-10
        y2 = int(0.5*frame.shape[1])
        cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
        cv2image = cv2.cvtColor(cv2image, cv2.COLOR_BGR2RGBA)
        current_image = Image.fromarray(cv2image)
        cv2image = cv2image[y1:y2, x1:x2]
        gray = cv2.cvtColor(cv2image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),2)
        th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
        cv2.imshow('th3',th3)
        ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        result=predict(res)
        cv2.putText(frame,result, (50, 50),cv2.FONT_HERSHEY_COMPLEX,2,(0,255,0),1)
        cv2.imshow("Image",frame)

    if(cv2.waitKey(1)==13):
        break         

cap.release()
cv2.destroyAllWindows()